In [1]:
import pandas as pd
import time
import numpy as np

In [2]:
csvfile = '../clean_data/cleaned_get_it_done_closed_potholes.csv'
df = pd.read_csv(csvfile)

In [ ]:
df = df.drop(['address', 'sap_notification_number', 'sap_problem_code', 'service_subtype', 'source', \
            'referred_email_update', 'referral_email', 'functional_location', 'description', \
            'agency_responsible' , 'sap_problem_category', 'case_record_type', 'updated_datetime',\
            'service_request_id', 'parent_case_number'], axis = 1)

In [3]:
#drop all potholes 
#df = df.drop(df[df.sap_problem_type != 'Pothole'].index)

#drop all NaNs for Long and Lat
df = df.dropna(subset = ['long'])
df = df.dropna(subset = ['lat'])

#drop all non-active potholes
df = df.drop(df[df.status_description == 'Duplicate'].index)
df = df.drop(df[df.status_description== 'Closed'].index)
df = df.drop(df[df.status_description=='Closed - Referred'].index)

#Drop all values that are open
df = df.dropna(subset = ['closed_date'])


In [4]:
neighborhoods = np.zeros((len(df['lat']),1))
df['Neighborhood'] = neighborhoods

# Vinh : 0-2400
# Lee Anne: 2400 - 4800
# Maggie: 4800-7200
# Jiggy: 7200 - 9600 
# Tim 9600 - 12000

In [5]:
#get neighborhoods of potholes
import geocoder 

#SET THE X AND Y
x = 2400
y = 4800
neighborhood = []

for i in range (x,y):
    latitutde = df.iloc[[i]]['lat']
    longitude = df.iloc[[i]]['long']
    time.sleep(.1)
    g = geocoder.google([latitutde, longitude], method = 'reverse').json
    if 'neighborhood' not in g:
         df.ix[i, 'Neighborhood'] = 'NaN'
    else:
        neighb = g['neighborhood']
        df.ix[i, 'Neighborhood'] = neighb
    
# for i in range (1401, len(df['long'])):
#     neighborhood.append('ADD ME')
# df['Neighborhood'] = neighborhood

# NAME THIS AS '../clean_data/0-2500' 
# or whatever you're range was

In [7]:
df.to_csv('../clean_data/closed_2400-4800.csv')

In [315]:
from datetime import datetime
date_format = "%Y-%m-%d %H:%M:%S"

In [316]:
for i in df.index:
    closed_date = str(df['closed_date'][i])
    open_date = str(df['requested_datetime'][i])
    closed_date = closed_date.replace('T',' ')
    open_date = open_date.replace('T',' ')
    df.ix[i, 'closed_date'] = closed_date
    df.ix[i, 'requested_datetime'] = open_date


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


'2017-05-23 08:41:00'

In [327]:
time_list = []

for i in df.index:
    a = datetime.strptime(str(df['closed_date'][i]), date_format)
    b = datetime.strptime(str(df['requested_datetime'][i]), date_format)
    delta = a - b
    hours_seconds = delta.seconds/3600
    hours_days = delta.days * 24
    time_list.append(hours_days + hours_seconds)
df['Duration to Close (Hrs)'] = time_list

In [326]:
a = datetime.strptime(str(df['closed_date'][6125]), date_format)
b = datetime.strptime(str(df['requested_datetime'][6125]), date_format)
delta = a - b
days = delta.minutes
print(a)
print(b)
print(days)

AttributeError: 'datetime.timedelta' object has no attribute 'minutes'

[0, 0, 0, 0, 0, 0, 0, 0, 0]